In [15]:
import pandas as pd
import numpy as np
from math import log2
from matplotlib import pyplot as plt
from scipy.sparse import diags
from sklearn.utils import as_float_array
import numpy.linalg as npl
import numpy.random as npr

%matplotlib inline

In [2]:
from sketching import datasets

In [3]:
dataset = datasets.Covertype_Sklearn()
Z = dataset.get_Z()
print(type(Z), Z.shape)

2021-09-27 15:52:54,842 - PID: 13930 - PName: MainProcess - INFO - Binary files for X and y found!
2021-09-27 15:52:54,843 - PID: 13930 - PName: MainProcess - INFO - Loading X from path /Users/anuprao/Documents/code/coreset_logistic_regression/LogReg_code/data/covertype_sklearn_X.npy
2021-09-27 15:52:54,995 - PID: 13930 - PName: MainProcess - INFO - Loading y from path /Users/anuprao/Documents/code/coreset_logistic_regression/LogReg_code/data/covertype_sklearn_y.npy
2021-09-27 15:52:55,356 - PID: 13930 - PName: MainProcess - INFO - Binary file for beta_opt found!
2021-09-27 15:52:55,357 - PID: 13930 - PName: MainProcess - INFO - Loading beta_opt from path /Users/anuprao/Documents/code/coreset_logistic_regression/LogReg_code/data/covertype_sklearn_beta_opt.npy
2021-09-27 15:52:55,359 - PID: 13930 - PName: MainProcess - INFO - Computing f_opt...
2021-09-27 15:52:55,788 - PID: 13930 - PName: MainProcess - INFO - Done.


<class 'numpy.ndarray'> (581012, 55)


In [19]:
def fast_QR(X, return_error=False):
    """
    Description:
        this calculates a fast approximation of the QR decomposition
    Parameter:
        X - np.array : the Matrix to decompose
    Return:
        Q - np.array : the Q part
    """
    # compress X into a approximation
    n, d = X.shape

    # mapping function to compress n x d - matrix into d**2 x d - matrix
    # f: {0,...,n-1} -> {0,...,d**2-1}
    f = npr.randint(d ** 2, size=n)
    # mapping function to determine the sign
    # g: {0,...,n-1} -> {-1,1}
    g = npr.randint(2, size=n) * 2 - 1

    # init the sketch
    X_ = np.zeros((d ** 2, d))
    for i in range(n):
        X_[f[i]] += g[i] * X[i]

    R_ = np.linalg.qr(X_, mode="r")
    error = False
    try:
        R_inv = np.linalg.inv(R_)
    except np.linalg.LinAlgError as err:
        print("LinAlgError: {0}".format(err), file=sys.stderr)
        print(
            "Error in fast_QR: R_ is not invertable, because singulare matrix!"
            + " continuing with pseudo invers."
        )
        R_inv = np.linalg.pinv(R_)
        error = True

    Q_ = np.dot(X, R_inv)

    if return_error:
        return Q_, error
    else:
        return Q_


def gauss_QR(X, k=1):
    """
    Description:
        this calculates a fast approximation of the QR decomposition with a
        gauss Vector
    Parameter:
        X - np.array : the Matrix to decompose
    Return:
        Q - np.array : the Q part
    """
    # compress X into a approximation
    n, d = X.shape

    # mapping function to compress n x d - matrix into d**2 x d - matrix
    # f: {0,...,n-1} -> {0,...,d**2-1}
    f = npr.randint(d ** 2, size=n)
    # mapping function to determine the sign
    # g: {0,...,n-1} -> {-1,1}
    g = npr.randint(2, size=n) * 2 - 1

    # init the sketch
    X_ = np.zeros((d ** 2, d))
    for i in range(n):
        X_[f[i]] += g[i] * X[i]

    R_ = np.linalg.qr(X_, mode="r")
    try:
        R_inv = np.linalg.inv(R_)
    except np.linalg.LinAlgError as err:
        print("LinAlgError: {0}".format(err), file=sys.stderr)
        print(
            "Error in gauss_QR: R_ is not invertable, because singulare matrix!"
            + " continuing with pseudo invers."
        )
        R_inv = np.linalg.pinv(R_)

    n, d = R_inv.shape
    g = np.random.normal(loc=0, scale=1 / np.sqrt(k), size=(d, k))
    # print(g)
    # print(str(R_inv.shape) + "//" + str(g.shape))
    r = np.dot(R_inv, g)
    # print(r)
    # print(str(X.shape) + "//" + str(r.shape))
    Q_ = np.dot(X, r)
    return Q_




In [21]:
def _calculate_lev_score_exact(X):
    Xt = X.T
    XXinv = np.linalg.pinv(Xt.dot(X))
    lev = np.zeros(Z.shape[0])
    for i in range(Z.shape[0]):
        xi = X[i:i+1,:]
        lev[i] = (xi.dot(XXinv)).dot(xi.T)
    return lev

def _calculate_lewis_weights_exact(X, qr,T=20):
    n = X.shape[0]
    w = np.ones(n)
    
    
    for i in range(T):
        assert(min(w)>0, min(w))
        Wp = diags(np.power(w,-0.5))
        #Q = qr(Wp.dot(X))
        #s = _calculate_sensitivities_leverage(Q) 
        s = _calculate_lev_score_exact(Wp.dot(X))
        w_nxt = np.power(w*s,0.5)
        print("|w_t - w_t+1|/|w_t| = ", np.linalg.norm(w-w_nxt)/np.linalg.norm(w))
        w = w_nxt
        
    return np.array(w+1.0/n, dtype=float)



<>:16: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:16: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-21-37d1e85c86ae>:16: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(min(w)>0, min(w))


In [31]:
def _calculate_sensitivities_leverage(Q, n=0):
    if n == 0:
        n = Q.shape[0]
    s = []

    for q in Q:
        s.append(npl.norm(q) ** 2)

    return np.array(s, dtype=float)

def _calculate_lewis_weights(X, qr, bet=0.1, T=20):
    n = X.shape[0]
    w = np.ones(n)
    
    
    for i in range(T):
        assert min(w)>0, str(min(w))
        Wp = diags(np.power(w,-0.5))
        Q = qr(Wp.dot(X))
        s = _calculate_sensitivities_leverage(Q) 
        w_nxt = np.power(w*s,0.5)
        print("|w_t - w_t+1|/|w_t| = ", npl.norm(w-w_nxt)/npl.norm(w))
        w = w_nxt
        
    return np.array(w+1.0/n, dtype=float)


def lewis_sampling(
    data,
    output_size_param=100,
    max_dim=0,
    return_sensitivities=False,
    type="normal",
    k=1,
    sensitivity_function=_calculate_lewis_weights,
):
    """Subsample data with replacement by L1 Lewis weights.

    Parameters
    ----------
    data : array-like matrix, shape=(n_samples, n_features)

    output_size_param : int or float, optional
        Default is 100.

    max_dim : int
        If positive, maximum number of features to use. If zero or negative,
        all features are used. Default is 0.

    return_sensitivities : boolean
        If True sensitivitie is returned as well. Default = False

    Returns
    -------
    coreset : float ndarray with shape (subsample_size, n_features)

    weights : float ndarray with shape(subsample_size,)

    s : float ndarray with shape(subsample_size,)
        sensitivitie of each entry in the coreset
    """
    assert len(data.shape) == 2

    num_samples, num_features = data.shape
    # checking parameter
    if isinstance(output_size_param, int):
        if output_size_param <= 0:
            raise ValueError(
                "If output_size_param is an int, it must be "
                "positive, but a value of %d was passed" % output_size_param
            )
        coreset_size = output_size_param
        if coreset_size > num_samples:
            coreset_size = num_samples
    elif isinstance(output_size_param, float):
        if output_size_param <= 0.0 or output_size_param > MAX_CORESET_PROPORTION:
            raise ValueError(
                "If output_size_param is a float, it must be "
                "greater than 0.0 and at most %.1f, but a value "
                "of %.2f was passed" % (MAX_CORESET_PROPORTION, output_size_param)
            )
        coreset_size = max(1, int(num_samples * output_size_param + 0.5))
    else:
        raise ValueError("'output_size_param' must be an int or float")
    if not isinstance(max_dim, int):
        raise ValueError("'max_dim' must be an int")

    D = as_float_array(data, copy=False)
    # creat a QR=D decomposition
    if type == "normal":
        qr = lambda A : npl.qr(A, mode="reduced")[0]
        #Q, R = npl.qr(D, mode="reduced")
    elif type == "sketch approx":
        qr = lambda A : fast_QR(A)
        #Q = fast_QR(D)
    elif type == "gauss approx":
        qr = lambda A : gauss_QR(A,k)
        #Q = gauss_QR(D, k)
    elif type == "exact":
        qr = "not a function"
        sensitivity_function=_calculate_lewis_weights_exact
    else:
        raise ValueError("type not correct")
    s = sensitivity_function(data, qr)

    # calculate probabilities
    p = s / np.sum(s)

    return p
#     coreset_indices = npr.choice(p.shape[0], size=coreset_size, p=p, replace=True)
#     index_weights = 1.0

#     # calculate the weight
#     weights = index_weights / (p[coreset_indices] * coreset_size)

#     if return_sensitivities:
#         return data[coreset_indices, :].copy(), weights, {"sensitivitie": s}
#     else:
#         return data[coreset_indices, :].copy(), weights

In [32]:
lewis_sampling(
    Z,
    output_size_param=100,
    max_dim=0,
    type="exact"
)

|w_t - w_t+1|/|w_t| =  0.9918949133749629
|w_t - w_t+1|/|w_t| =  0.8666049811637239
|w_t - w_t+1|/|w_t| =  0.47490087107364254
|w_t - w_t+1|/|w_t| =  0.19033850226819574
|w_t - w_t+1|/|w_t| =  0.08155279914647695
|w_t - w_t+1|/|w_t| =  0.037702509071291794
|w_t - w_t+1|/|w_t| =  0.018141719621905843
|w_t - w_t+1|/|w_t| =  0.008902248184720846
|w_t - w_t+1|/|w_t| =  0.004410564430348993
|w_t - w_t+1|/|w_t| =  0.002195580818463185
|w_t - w_t+1|/|w_t| =  0.0010955364736440995
|w_t - w_t+1|/|w_t| =  0.000547283277999639
|w_t - w_t+1|/|w_t| =  0.0002735578415147865
|w_t - w_t+1|/|w_t| =  0.00013677605548034853
|w_t - w_t+1|/|w_t| =  6.839613505342425e-05
|w_t - w_t+1|/|w_t| =  3.4204360131374e-05
|w_t - w_t+1|/|w_t| =  1.710594512491513e-05
|w_t - w_t+1|/|w_t| =  8.55479464718036e-06
|w_t - w_t+1|/|w_t| =  4.278568659905558e-06
|w_t - w_t+1|/|w_t| =  2.1396992001513424e-06


array([1.03008072e-06, 1.09827602e-06, 9.06303423e-07, ...,
       2.68606103e-06, 2.68240355e-06, 2.67477685e-06])

In [34]:
lewis_sampling(
    Z,
    output_size_param=100,
    max_dim=0,
    type="sketch approx"
)

|w_t - w_t+1|/|w_t| =  0.9915833995493268
|w_t - w_t+1|/|w_t| =  0.8677505172835166
|w_t - w_t+1|/|w_t| =  0.4773284579931527
|w_t - w_t+1|/|w_t| =  0.1906407245082601
|w_t - w_t+1|/|w_t| =  0.08192740723158992
|w_t - w_t+1|/|w_t| =  0.04025621714484131
|w_t - w_t+1|/|w_t| =  0.0179892321407858
|w_t - w_t+1|/|w_t| =  0.008784930876675007
|w_t - w_t+1|/|w_t| =  0.004991914588022856
|w_t - w_t+1|/|w_t| =  0.0056787339986027665
|w_t - w_t+1|/|w_t| =  0.004268674790248996
|w_t - w_t+1|/|w_t| =  0.004870354484221225
|w_t - w_t+1|/|w_t| =  0.004217344732823587
|w_t - w_t+1|/|w_t| =  0.005613884907381419
|w_t - w_t+1|/|w_t| =  0.004735348819976428
|w_t - w_t+1|/|w_t| =  0.004760587310434089
|w_t - w_t+1|/|w_t| =  0.006392056861079647
|w_t - w_t+1|/|w_t| =  0.004537156500235937
|w_t - w_t+1|/|w_t| =  0.007682038423135265
|w_t - w_t+1|/|w_t| =  0.004960612577451337


array([1.03474233e-06, 1.11146276e-06, 9.24659152e-07, ...,
       2.69567094e-06, 2.69062047e-06, 2.67886047e-06])